# Informe de Prácticas: Técnicas de Algoritmos de Búsqueda

## Introducción

El objetivo de esta práctica ha sido profundizar en técnicas de algoritmos de búsqueda aplicadas al problema de correspondencia de grafos (graph matching). El trabajo se ha centrado principalmente en la visualización, construcción y matching de grafos basados en puntos clave (keypoints) de imágenes del conjunto de datos Willow-ObjectClass.

Este conjunto de datos contiene imágenes de cinco categorías (car, duck, face, motorbike y winebottle), cada una anotada con keypoints específicos. El objetivo principal ha sido implementar y comparar diferentes algoritmos para establecer correspondencias entre keypoints de pares de imágenes, evaluando su precisión y rendimiento.

He estructurado el trabajo en varios módulos, dentro del directorio principal del programa encontramos las subcarpetas `data` y `results` y los notebooks de cada bloque requerido. Todo esto se detalla más adelante.

## DataLoader

Para facilitar el trabajo con los datos, decidí crear una clase externa llamada `DataLoader` que se encarga de leer y organizar todas las carpetas de datos, se encuentra dentro de la carpeta `data` y este directorio funciona como una libreria. Esta clase escanea el directorio base (por defecto, la carpeta 'WILLOW-ObjectClass' del proyecto) y busca imágenes `.png` junto con sus correspondientes archivos `.mat` de keypoints. Guarda los paths de las imágenes y keypoints, cada categoría en un dataframe separado. La función `load_data` se encarga de hacer todo esto y devolver un diccionario con los dataframes de cada categoría. Cada dataframe contiene las columnas 'image' y 'keypoints', con los paths de las imágenes y keypoints respectivamente.

Esta clase ha sido imprescindible para mantener el código limpio y modular a lo largo de todas las partes de la práctica, evita la duplicación de código y proporciona un acceso facil a todos los datos.

## Visualización de Imágenes y Keypoints - Parte 1

El objetivo de esta primera parte de la práctica fue la visualización básica de las imágenes y sus keypoints. Para ello configuré correctamente el PYTHONPATH para poder importar el módulo DataLoader, cosa que hice en cada notebook, es muy importante para poder importar la clase DataLoader. Cargué las imágenes y sus correspondientes keypoints. Redimensioné todas las imágenes a un tamaño uniforme de 256x256 píxeles. Ajusté las coordenadas de los keypoints para mantener su correspondencia con las imágenes redimensionadas. Finalmente visualicé los conjuntos de 8 imágenes aleatorias por categoria en grids de 2x4 con sus respectivos keypoints superpuestos


La función `load_img_and_keypoints_from_row` resultó super útil, ya que abstrae todo el proceso de carga y redimensionamiento, permitiéndome reutilizarla en las siguientes partes de la práctica. Al repetirla tantas veces en las partes de la práctica posteriores, me aprendí de memoria como buscar y redimensionar las imágenes, ajustar los keypoints y devolver todo redimensionado.

## Visualización de Imágenes y Keypoints - Parte 2

En esta segunda parte de la visualización se basaba en la construcción de grafos a partir de los keypoints. Para esto usamos dos métodos diferentes:

### Triangulación de Delaunay

La triangulación de Delaunay es una técnica geométrica que conecta puntos formando triángulos uniformes dentro de un grafo. Decidí implementar este método de forma manual, sin utilizar libreria Delaunay. Esto lo hice para entender mejor su funcionamiento desde el más bajo nivel. La función usaba 3 bucles anidados para iterar sobre todos los puntos y comprobar si formaban un triángulo válido. Realmente pensaba que iba a ser más dificil de implementar, estos puntos en los que iteraban los bucles hacian operaciones matemáticas sencillas como calcular la distancia entre dos puntos, comprobar si un punto estaba dentro de un triángulo, incluir en una lista los puntos validos, etc.

- pd(for ahmed): Este método creo que se usa mucho en tecnicas de reescalado de objetos 3D, generación de mallas en blender, etc. Supongo que existirán varias tecnicas de triangulación, no se si conoces Nanite, pero creo que es la tecnica de triangulación más avanzada para objetos 3D que existe actualmente. La usaba hace tiempo cuando hacia algún que otro juego en Unreal Engine 5.

### Grafos K-NN (K vecinos más cercanos)

Este otro tipo de técnica de triangulación se basa en la construcción de grafos K-NN, donde cada punto se conecta con sus K vecinos más cercanos. Esta función es bastante más sencilla que la triangulación de Delaunay, simplemente calcula las distancias entre todos los puntos y selecciona los K más cercanos, cuantos mas K más conexiones se generan. Esto se puede ver con los valores K (3, 5 y 7) que tomé para cada categoría de imagen, lo que permitió observar cómo la densidad de conexiones afecta a la estructura del grafo resultante.

La visualización de ambos métodos nos permite compararlos y entender sus diferencias: mientras que Delaunay genera una triangulación uniforme, K-NN tiende a crear más conexiones en áreas donde los puntos están más concentrados.

## Generación de Grafos Matcheados - Parte 1

Esta parte de la practica tenía como objetivo generar las correspondencias entre pares de grafos utilizando los grafos construidos anteriormente. El objetivo es encontrar qué keypoints de una imagen se corresponden con los de otra imagen de la misma categoría.

En esta parte he implementado tres funciones de `carga y plotteo de imagenes`, la función de `triangulacion Delunay`, para visualizar las triangulaciones de las imagenes, la función de `matching` con el uso del algoritmo húngaro que realiza un matching basandose en las distancias euclidianas entre los keypoints y finalmente las funciones que evaluan y generan las precisiones del matching para todas las categorías de imágenes.

Indagaré un poco más en el algoritmo húngaro, y en las tres funciones de carga y plotteo de imagenes y kaypoints. `Load_img_and_keypoints_from_row` genera las imagenes y keypoints redimensionados y triangulados, esta nos va a servir solamente para visualizar las imagenes y keypoints de las categorias. Para hacer los matchings finales con el algoritmo húngaro, se usará la función de `load_keypoints_from_row` que solamente tiene en cuenta los keypoints de las imagenes ya que el algoritmo húngaro solo necesita los keypoints para hacer el matching, esto hará nuestro codigo más eficiente que si solo usaramos la función de `load_img_and_keypoints_from_row`. Finalmente, usaremos la función de `visualize_combined` que nos generará la visualización de las imagenes con los grafos  

Las correspondencias correctas se visualizan con líneas verdes y las incorrectas con líneas rojas, permitiendo evaluar visualmente la calidad del matching. Este enfoque básico, basado únicamente en coordenadas espaciales, funciona razonablemente bien para imágenes similares pero tiene limitaciones cuando las posiciones de los objetos varían significativamente.

## Generación de Grafos Matcheados - Parte 2

En la parte final y más avanzada, mejoré el algoritmo de matching incorporando características topológicas además de las espaciales. Las principales innovaciones fueron:

1. **Implementación de Node2Vec**: Esta técnica genera embeddings vectoriales para cada nodo del grafo que capturan información sobre su contexto estructural y su posición en la red. Los nodos con roles similares en distintos grafos tendrán embeddings similares.

2. **Tiempos de llegada (Hitting Times)**: Calculé matrices que representan el número esperado de pasos que tarda un paseo aleatorio en llegar de un nodo a otro, capturando información global sobre la conectividad del grafo.

3. **Matching mejorado**: Combiné las características espaciales con las topológicas para crear una matriz de costes ponderada, balanceando la importancia de la posición espacial y la estructura del grafo.

Además, implementé un sistema de evaluación automática que calcula la precisión de los matchings para todas las categorías de imágenes, generando estadísticas como la precisión media y la desviación estándar para cada categoría.

También experimenté con distintas configuraciones de pesos para las diferentes características, analizando cómo afectaban a la precisión final del matching.

## Evaluación de Resultados

Según los resultados obtenidos y guardados en los archivos CSV, puedo concluir que:

1. La incorporación de características topológicas mejoró significativamente la precisión del matching en todas las categorías, como se puede observar comparando results.csv (enfoque espacial básico) con results2.csv (enfoque mejorado).

2. Las categorías "face" y "winebottle" obtuvieron los mejores resultados de matching, con precisiones del 90.92% y 89.31% respectivamente en el enfoque mejorado.

3. La categoría "duck" presentó la menor precisión (70.72%), probablemente debido a que los keypoints son más ambiguos o la variabilidad entre imágenes es mayor.

4. El método mejorado no sólo incrementó la precisión media, sino que también redujo la desviación estándar en todas las categorías, lo que indica un comportamiento más consistente.

En conclusión, este trabajo me ha permitido implementar y comprender diversas técnicas de correspondencia de grafos, desde los enfoques más básicos hasta métodos avanzados que incorporan información topológica. Los resultados demuestran claramente que considerar la estructura del grafo, además de la posición espacial de los keypoints, mejora significativamente la capacidad de establecer correspondencias correctas entre imágenes.